<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction


## How to use this notebook
This example runs Sima in the cloud based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. This notebook assumes that all tooling is in place see the installation section of the notebook for [local execution](workflowCoreDemoSE28SestraAndWasim.ipynb#installation)

This note book should be run in the following way:
1. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
2. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
3. [Run analysis](#run) shall be run each time a new analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.


# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up custom user parameters <a id='custom'></a>

In [ ]:
from pathlib import Path
import os
import shutil
oneWorkflowTMPFolder = r'd:\oneworkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
        print(oneWorkflowTMPFolder + " created!\n")
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath
root_folder = os.getcwd()
print(root_folder)
oldworkspacePath = str(Path(root_folder, 'Workspace'))
workspacePath = str(Path(root_folder, 'WorkspaceCloud'))
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloudRun = False
notebook_root_folder = os.getcwd()
shutil.copytree(oldworkspacePath, workspacePath,dirs_exist_ok=True, ignore=shutil.ignore_patterns("LoadCases"))

## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [7]:
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils import *
#workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder,
#                                      local_workflow_runtime_temp_folders_cleanup=False,platform=Platform.LINUX, max_cores=5,environment=Environment.Testing,pool_id="SesamWorkflow_Linux_Standard_A1_v2")

workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder,
                                      local_workflow_runtime_temp_folders_cleanup=False,platform=Platform.WINDOWS, max_cores=5,environment=Environment.Testing,pool_id="SesamWorkflow_Windows_Standard_A2_v2")


The temporary blob storage directory is: d:\oneworkflowTmp\oc_2fcm6o1i_blob
The temporary jobs root directory is: d:\oneworkflowTmp\oc_ajvd4hfq_jobs


# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [ ]:
from dnv.sesam.sima_command import SimaCommand
import pandas as pd
"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

commands_info = []

df_cases = pd.read_excel(loadcase_file, index_col=0)

for loadcase_folder_name, case in df_cases.iterrows():
    sima_cmd = SimaCommand().create_sima_command(task_name="WorkflowTask",workflow_name="ExampleWorkflow",sima_case_data=case.to_dict(),stask_filename=stask_file,stask_foldername=workflow_client.common_directory)
    commands_info.append(CommandInfo(commands=[sima_cmd],load_case_foldername=loadcase_folder_name))
print("Running commands in parallel")
await run_managed_commands_parallelly_async(
            log_job = True,
            client=workflow_client,
            commands_info=commands_info,
            files_to_download_from_blob_to_client=FileOptions(max_size="11124MB",patterns=["**/*.log","**/*.txt", "**/*.lis", "**/*.MLG", "**/*.MLG","**/*.CSV"]),
            enable_common_files_copy_to_load_cases=True,
)




# Run Wasim and Sestra <a id='runwasim'></a>

In [ ]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.sesam.sestra_command import SestraCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
import json
import os
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]
windows_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, "", Platform.Windows)
os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue

        casedict = case.to_dict()
       
        load_case_result_files_dir = os.path.join(results_directory, casename)
        cmds = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        cmd = CompositeExecutableCommand(cmds)
        work_unit = (
            WorkUnit(cmd,  f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"
                                                                         ,"**/*.LIS", "**/*.MLG"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)

from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
import json
import os
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def create_wasim_and_sestra_tasks(load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    commands_info = []
    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        casedict = case.to_dict()
        tasks = WasimAndSestraTaskCreator(casedict, additionalTemplateParameters).CreateTasks()
        cmd_info = CommandInfo(commands=tasks,load_case_foldername=casename)
        commands_info.append(cmd_info)
       

    return commands_info
print(os.getcwd())
print(workflow_client.workspace_path)

windows_client.login()
work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)

downloadOptions = FileOptions(
    #max_size_bytes=1001124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
#await workflow_windows_client.run_workflow_async(
#    work_unit,
    #work_item_status_changed=work_item_status_changed_callback(
        #workflow_windows_client, downloadOptions),
    #log_job=True
#)




In [ ]:
await run_workflow_async_two_clients(
    job, workflow_client, windows_client, work_unit, option=downloadOptions)


# Post processing  <a id='postprocessing'></a>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
print(workspacePath)
for folder in glob.glob(f"{workspacePath}\\Results\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)


def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(
    multiplot, resultcase=dataFrames.keys())
interactive_plot
